In [13]:
import mlx.core as mx
from tqdm import tqdm
from PIL import Image
import numpy as np

import mlx.nn as nn
from stable_diffusion import StableDiffusion

In [14]:
def create_gif(image_array, output_path, fps=10):
    """
    Create a GIF from an array of images.

    Parameters:
    - image_array: numpy array of shape (num_images, height, width, channels)
    - output_path: string, the file path to save the output GIF
    - fps: int, frames per second
    """
    duration = int(1000 / fps)  # Convert fps to duration in milliseconds per frame
    frames = [Image.fromarray(image_array[i].astype('uint8')) for i in range(image_array.shape[0])]
    frames[0].save(output_path, save_all=True, append_images=frames[1:], duration=duration, loop=0)

In [15]:
sd = StableDiffusion("stabilityai/stable-diffusion-2-1-base",float16=True)
nn.quantize(sd.text_encoder, class_predicate=lambda _, m: isinstance(m, nn.Linear))
nn.quantize(sd.unet, group_size=32, bits=8)

noises = sd._init_walk_noise(walk_steps=400,size=2)


In [16]:
images=[]

for walk_step in tqdm(range(noises.shape[0])):
    latent_generator = sd.generate_latents_from_noise(
        "bunny in a knife with a dark scary",
        noises[walk_step],
        num_steps=10)
    for x_t in latent_generator:
        mx.eval(x_t)
    im = sd.decode(x_t)
    images.append(im)

100%|██████████| 400/400 [1:12:35<00:00, 10.89s/it]


In [17]:
title="poop3"
imagesnp = np.array(images)
imagesnp = np.squeeze(imagesnp,axis=1)
imagesnp = (imagesnp * 255).astype(np.uint8)
create_gif(imagesnp,f"{title}.gif",20)